In [1]:
from os import getcwd, makedirs
from os.path import join, abspath, exists

from copy import deepcopy
from joblib import Parallel, delayed, parallel_backend
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from tqdm import tqdm

from src.models.estimator import RFClassifier, relu
from src.models.weights import V1_weights, classical_weights
from src.data.load_dataset import load_mnist
from sklearn.linear_model import LogisticRegression

/home/whitev4/anaconda3/envs/random_features/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


In [2]:
data_dir = abspath(join(getcwd(), '../../'))

In [3]:
# load dataset
train_loader, val_loader, test_loader = load_mnist()
train, test = train_loader.dataset.dataset, test_loader.dataset
train, train_labels = train.data.numpy(), train.targets.numpy()
X_test, y_test = test.data.numpy(), test.targets.numpy()

X_train, X_val, y_train, y_val = train_test_split(train, train_labels, train_size=0.999, stratify=train_labels)

In [4]:
num_neurons = 100
num_trials = 5
nonlinearity = relu
bias = 0
scale = 1
clf = LinearSVC(tol=1e-4, max_iter=1000)
n_jobs=3

# V1 params
compatible = {'s': 5, 'f':2}
incompatible = {'s': 0.5, 'f':0.5}

### V1 weights RFNet + optimal parameters

In [5]:
s, f = compatible['s'], compatible['f']
kwargs = {'size':s, 'spatial_freq':f, 'center':None, 'scale':scale}

clf1 = RFClassifier(num_neurons, V1_weights, bias, nonlinearity, deepcopy(clf), kwargs);
clf1.fit(X_train, y_train)
clf1.score(X_val, y_val)

/home/whitev4/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9333333333333333

In [6]:
clf1.score(X_test, y_test)

0.9123

### Classical weights RFNet

In [7]:
kwargs = {'scale':scale}
clf2 = RFClassifier(num_neurons, classical_weights, bias, nonlinearity, deepcopy(clf), kwargs);
clf2.fit(X_train, y_train)
clf2.score(X_val, y_val)

/home/whitev4/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.8166666666666667

In [8]:
clf2.score(X_test, y_test)

0.8205

### "Incompatible" weights RFNet

In [9]:
s, f = incompatible['s'], incompatible['f']
kwargs = {'size':s, 'spatial_freq':f, 'center':None, 'scale':scale}
clf3 = RFClassifier(num_neurons, V1_weights, bias, nonlinearity, deepcopy(clf), kwargs)
clf3.fit(X_train, y_train)
clf3.score(X_val, y_val)

/home/whitev4/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.7

In [10]:
clf3.score(X_test, y_test)

0.7163